In [1]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [2]:
f5_inputs = np.load("initial_inputs.npy")
f5_outputs = np.load("initial_outputs.npy")
f5_inputs_2 = np.load("initial_inputs2.npy")
f5_outputs_2 = np.load("initial_outputs2.npy")

In [3]:
f5_inputs = np.concatenate((f5_inputs, f5_inputs_2)) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.329622, 0.319559, 0.335951, 0.275429]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.630732, 0.255243, 0.163612, 0.663668]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.136884, 0.175429, 0.750162, 0.240518]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.156884, 0.185429, 0.730162, 0.834869]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.221966, 0.848702, 0.877261, 0.876293]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.2353  , 0.835369, 0.890595, 0.889626]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.255301, 0.815369, 0.910595, 0.909626]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.275301, 0.795369, 0.930595, 0.929626]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.295301, 0.775369, 0.950595, 0.949626]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.30362 , 0.92081 , 0.951794, 0.890271]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.30362 , 0.956366, 0.956239, 0.872493]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.288064, 0.967477, 0.954016, 0.856937]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.305842, 0.976366, 0.962905, 0.87027]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.290286, 0.996366, 0.982905, 0.890271]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.283619, 0.996366, 0.994016, 0.905826]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.285841, 0.987477, 0.994016, 0.925826]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.283618, 0.985544, 0.999999, 0.936937]]))) 
f5_inputs = np.concatenate((f5_inputs, np.array([[0.281395, 0.978877, 0.999999, 0.956937]]))) 

f5_outputs = np.concatenate((f5_outputs,f5_outputs_2))
f5_outputs = np.append(f5_outputs,131.1962333718706)
f5_outputs = np.append(f5_outputs,8.319139067926972)
f5_outputs = np.append(f5_outputs,15.916258413112816)
f5_outputs = np.append(f5_outputs,65.62143113848653)
f5_outputs = np.append(f5_outputs,1077.2793665718714)
f5_outputs = np.append(f5_outputs,1151.2287899694788)
f5_outputs = np.append(f5_outputs,1283.9813760530844)
f5_outputs = np.append(f5_outputs,1446.3325506169003)
f5_outputs = np.append(f5_outputs,1642.712076836066)
f5_outputs = np.append(f5_outputs,2043.5758440370957)
f5_outputs = np.append(f5_outputs,2246.9353067103452)
f5_outputs = np.append(f5_outputs,2221.1489548557843)
f5_outputs = np.append(f5_outputs,2475.0054048920615)
f5_outputs = np.append(f5_outputs,3030.1696296258147)
f5_outputs = np.append(f5_outputs,3286.085078568141)
f5_outputs = np.append(f5_outputs,3367.83461018011)
f5_outputs = np.append(f5_outputs,3526.9798701929503)
f5_outputs = np.append(f5_outputs,3659.8420356222346)


In [4]:
variable_value = np.linspace(1, 10,10)
n_dims = 4
beta =0.001# 1.1818181818181819
np.random.seed(42)
train_X = torch.tensor(f5_inputs)
train_Y = torch.tensor(f5_outputs).reshape(len(f5_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.001#  100 #1.2727272727272727
candidate_list = []
best_x_list = []

for i in range(len(variable_value)):
    #noise_level = variable_value[i]
   
    def function_7(x1,x2,x3,x4, noise_level=noise_level):
            return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
            np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
              + np.random.randn() * noise_level

    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    
    EI = qExpectedImprovement(model=gp , best_f=best_y)

    #UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=50, raw_samples=200,
    )


    x1,x2,x3,x4 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_7(x1,x2,x3,x4)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [5]:
best_x_list

[tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64),
 tensor([0.2814, 0.9789, 1.0000, 0.9569], dtype=torch.float64)]